<a href="https://colab.research.google.com/github/bibleme/Aiary/blob/one_line_diary/KoBART_synthetic_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 1. 먼저 실행: NumPy 버전을 1.x대로 강제 고정
!pip install "numpy<2.0"

# 2. 그 다음 나머지 라이브러리 설치
!pip install -q "transformers>=4.46.0,<5" sentencepiece accelerate datasets evaluate rouge-score

In [ ]:
# ================================
# 한 줄 일기 → 하루일기 KoBART 파인튜닝
# ================================

from google.colab import drive
drive.mount('/content/drive')

!pip install -q "transformers>=4.46.0,<5" sentencepiece accelerate datasets evaluate rouge-score

import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset

from transformers import (
    BartForConditionalGeneration,
    PreTrainedTokenizerFast,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)

import evaluate
import matplotlib.pyplot as plt

# --------------------------
# 경로 및 상수 설정
# --------------------------
BASE_DIR = "/content/drive/MyDrive/aiary"

MERGED_CSV = os.path.join(
    BASE_DIR,
    "data/parenting_dataset_v4/final_merged_dataset.csv"
)

OUTPUT_DIR = os.path.join(
    BASE_DIR,
    "models/day_diary_from_summary_v2"
)
os.makedirs(OUTPUT_DIR, exist_ok=True)

BASE_MODEL_NAME = "gogamza/kobart-base-v2"  # 쓰는 KoBART 모델명으로 바꿔도 됨
MAX_INPUT_LEN = 256    # summary 길이 그렇게 안 길어서 256 정도
MAX_TARGET_LEN = 384   # 하루일기 300~400자 정도 가정
RANDOM_SEED = 42

# --------------------------
# 1) 데이터 로드
#    merged.csv: id, source, diary, summary
# --------------------------
assert os.path.exists(MERGED_CSV), f"MERGED_CSV 파일 없음: {MERGED_CSV}"
df = pd.read_csv(MERGED_CSV)

required_cols = {"id", "source", "diary", "summary"}
assert required_cols.issubset(df.columns), f"CSV 컬럼 부족: {df.columns}"

df["diary"] = df["diary"].astype(str).str.strip()
df["summary"] = df["summary"].astype(str).str.strip()
df = df[(df["diary"] != "") & (df["summary"] != "")].reset_index(drop=True)

print(f"전체 샘플 수: {len(df)}")
print(df["source"].value_counts())

# --------------------------
# 2) train / val / test 분할
#    - val/test는 real만
# --------------------------
df_real = df[df["source"] == "real"].reset_index(drop=True)
df_synth = df[df["source"] == "synthetic"].reset_index(drop=True)

print(f"real 샘플 수: {len(df_real)}")
print(f"synthetic 샘플 수: {len(df_synth)}")

real_train, real_temp = train_test_split(
    df_real,
    test_size=0.3,
    random_state=RANDOM_SEED,
    shuffle=True,
)
real_val, real_test = train_test_split(
    real_temp,
    test_size=0.5,
    random_state=RANDOM_SEED,
    shuffle=True,
)

train_df = pd.concat([real_train, df_synth], ignore_index=True)
train_df = train_df.sample(frac=1.0, random_state=RANDOM_SEED).reset_index(drop=True)

val_df = real_val.reset_index(drop=True)
test_df = real_test.reset_index(drop=True)

print("\n분할 결과")
print(f"Train: {len(train_df)} (real={len(real_train)}, synthetic={len(df_synth)})")
print(f"Val:   {len(val_df)} (real only)")
print(f"Test:  {len(test_df)} (real only)")

# --------------------------
# 3) Dataset 정의
# --------------------------
class SummaryToDiaryDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len=256, max_target_len=384):
        self.df = df
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        summary = str(row["summary"])
        diary = str(row["diary"])

        # 인풋: 한 줄 일기 세트
        input_text = f"[SUMMARY]\n{summary}\n[DIARY]"
        target_text = diary

        model_inputs = self.tokenizer(
            input_text,
            max_length=self.max_input_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                target_text,
                max_length=self.max_target_len,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            )

        input_ids = model_inputs["input_ids"].squeeze(0)
        attention_mask = model_inputs["attention_mask"].squeeze(0)
        labels_ids = labels["input_ids"].squeeze(0)

        labels_ids[labels_ids == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels_ids,
        }



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
전체 샘플 수: 512
source
synthetic    300
real         212
Name: count, dtype: int64
real 샘플 수: 212
synthetic 샘플 수: 300

분할 결과
Train: 448 (real=148, synthetic=300)
Val:   32 (real only)
Test:  32 (real only)


In [ ]:
# --------------------------
# 4) 토크나이저 / 모델 로드
# --------------------------
print("\n[INFO] 토크나이저 / 모델 로드 중...")
tokenizer = PreTrainedTokenizerFast.from_pretrained(BASE_MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"device = {device}")

# --------------------------
# 5) Dataset, Collator, Trainer
# --------------------------
train_dataset = SummaryToDiaryDataset(train_df, tokenizer, MAX_INPUT_LEN, MAX_TARGET_LEN)
val_dataset = SummaryToDiaryDataset(val_df, tokenizer, MAX_INPUT_LEN, MAX_TARGET_LEN)
test_dataset = SummaryToDiaryDataset(test_df, tokenizer, MAX_INPUT_LEN, MAX_TARGET_LEN)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,          # 너무 오래 돌리면 오버피팅, 일단 5epoch
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=20,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
    seed=RANDOM_SEED,
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# --------------------------
# 6) 학습
# --------------------------
print("\n[INFO] 학습 시작...")
train_result = trainer.train()
trainer.save_state()

# --------------------------
# 7) 모델 / 토크나이저 저장
# --------------------------
print("\n[INFO] 모델 저장 중...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

with open(os.path.join(OUTPUT_DIR, "train_result.json"), "w", encoding="utf-8") as f:
    json.dump(train_result.metrics, f, ensure_ascii=False, indent=2)

# --------------------------
# 8) 학습 곡선 시각화
# --------------------------
log_history = trainer.state.log_history
train_steps, train_losses = [], []
eval_steps, eval_losses = [], []

for log in log_history:
    if "loss" in log and "step" in log:
        train_steps.append(log["step"])
        train_losses.append(log["loss"])
    if "eval_loss" in log and "step" in log:
        eval_steps.append(log["step"])
        eval_losses.append(log["eval_loss"])

if train_steps:
    plt.figure()
    plt.plot(train_steps, train_losses, label="train_loss")
    if eval_steps:
        plt.plot(eval_steps, eval_losses, label="eval_loss")
    plt.xlabel("step")
    plt.ylabel("loss")
    plt.title("Training & Validation Loss (Summary→Diary)")
    plt.legend()
    plt.grid(True)

    curve_path = os.path.join(OUTPUT_DIR, "training_curves.png")
    plt.savefig(curve_path, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"[INFO] 학습 곡선 저장: {curve_path}")
else:
    print("[WARN] train loss 로그가 없습니다.")



[INFO] 토크나이저 / 모델 로드 중...


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


device = cuda


/tmp/ipython-input-4020037289.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



[INFO] 학습 시작...


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.225800,2.856390
2,1.831500,2.791125
3,1.510200,2.821344
4,1.207500,2.866042
5,1.056700,2.897255


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__ca


[INFO] 모델 저장 중...
[INFO] 학습 곡선 저장: /content/drive/MyDrive/aiary/models/day_diary_from_summary_v2/training_curves.png


In [ ]:
# --------------------------
# 9) [수정됨] 한 샘플에 대해 하루일기 생성 함수 (KoBART 전용)
# --------------------------
def generate_diary_from_summary(summary_text: str, max_len: int = MAX_TARGET_LEN) -> str:
    input_text = f"[SUMMARY]\n{summary_text}\n[DIARY]"
    enc = tokenizer(
        input_text,
        max_length=MAX_INPUT_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

    # 필요한 애들만 꺼내서 device로 올리기
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=256,
            min_length=50,
            repetition_penalty=2.5,
            no_repeat_ngram_size=3,
            do_sample=True,
            temperature=0.5,
            top_p=0.85,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
        )

    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred = pred.replace("[DIARY]", "").strip()
    return pred

# --------------------------
# 10) 테스트 셋 예시 3개 출력
# --------------------------
print("\n[INFO] 테스트 셋 예시 3개 출력 (summary → diary)")

for i in range(min(3, len(test_df))):
    sample = test_df.iloc[i]
    summary_text = sample["summary"]
    diary_gold = sample["diary"]

    pred = generate_diary_from_summary(summary_text)

    print("=" * 80)
    print(f"[Sample {i+1}]")
    print("[SUMMARY (INPUT)]")
    print(summary_text)
    print("\n[GROUND TRUTH DIARY]")
    print(diary_gold)
    print("\n[PREDICTED DIARY]")
    print(pred)
    print("=" * 80)

# --------------------------
# 11) 테스트셋 전체 평가지표 (ROUGE_L만 간단히)
#      - 생성은 다양성이 커서 ROUGE는 참고용 정도로만
# --------------------------

# --------------------------
# 11) 테스트셋 Perplexity 계산
# --------------------------
print("\n[INFO] 테스트셋 Perplexity 계산 중...")

test_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_eval_batch_size=4,
    report_to=[],
)

# loss-only evaluation
test_trainer = Trainer(
    model=model,
    args=test_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

test_metrics = test_trainer.evaluate()

test_loss = test_metrics["eval_loss"]
test_ppl = torch.exp(torch.tensor(test_loss)).item()

print(f"[TEST LOSS] {test_loss:.4f}")
print(f"[TEST PPL]  {test_ppl:.4f}")

with open(os.path.join(OUTPUT_DIR, "test_perplexity.json"), "w", encoding="utf-8") as f:
    json.dump(
        {"test_loss": float(test_loss), "test_ppl": float(test_ppl)},
        f,
        ensure_ascii=False,
        indent=2,
    )

print("\n[INFO] 테스트셋 전체 ROUGE_L 계산 중...")

rouge = evaluate.load("rouge")

all_preds = []
all_refs = []

for i in range(len(test_df)):
    sample = test_df.iloc[i]
    summary_text = sample["summary"]
    diary_gold = sample["diary"]

    pred = generate_diary_from_summary(summary_text)
    all_preds.append(pred)
    all_refs.append(diary_gold)

rouge_result = rouge.compute(
    predictions=all_preds,
    references=all_refs,
    use_stemmer=True,
)

print("\n[TEST METRICS] (참고용)")
print(f"ROUGE-L: {rouge_result['rougeL']:.4f}")

metrics_path = os.path.join(OUTPUT_DIR, "test_metrics_rougeL.json")
with open(metrics_path, "w", encoding="utf-8") as f:
    json.dump(
        {"rougeL": rouge_result["rougeL"]},
        f,
        ensure_ascii=False,
        indent=2,
    )

print(f"\n[INFO] 테스트 평가지표 저장: {metrics_path}")
print("\n[INFO] 전체 파이프라인 완료")
print(f"모델/토크나이저/그래프 저장 위치: {OUTPUT_DIR}")

# --------------------------
# 12) 테스트셋 전체 생성 결과 저장 (CSV + JSONL)
# --------------------------
print("\n[INFO] 테스트셋 전체 생성 결과 저장 중...")

test_results = []

for i in range(len(test_df)):
    row = test_df.iloc[i]
    summary_text = row["summary"]
    diary_gold = row["diary"]

    pred = generate_diary_from_summary(summary_text)

    test_results.append({
        "id": row["id"],
        "summary": summary_text,
        "gold_diary": diary_gold,
        "predicted_diary": pred
    })

# CSV 저장
results_csv_path = os.path.join(OUTPUT_DIR, "test_generated_results.csv")
pd.DataFrame(test_results).to_csv(results_csv_path, index=False, encoding="utf-8-sig")

# JSONL 저장
results_jsonl_path = os.path.join(OUTPUT_DIR, "test_generated_results.jsonl")
with open(results_jsonl_path, "w", encoding="utf-8") as f:
    for r in test_results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"[INFO] CSV 저장 완료: {results_csv_path}")
print(f"[INFO] JSONL 저장 완료: {results_jsonl_path}")



[INFO] 테스트 셋 예시 3개 출력 (summary → diary)
[Sample 1]
[SUMMARY (INPUT)]
1. 딸이 방 안에서 불만스러운 표정을 짓고 있다. 2. 엄마가 청소한 방이 정돈되어 있지만, 딸의 마음은 복잡해 보인다. 3. 두 사람의 대화가 따뜻한 분위기를 만들어낸다.

[GROUND TRUTH DIARY]
오늘 딸이 나한테 실망했다고 하더라. 방 청소를 해줬는데도 불만이 많아서 좀 속상했어. 그래도 딸이 나를 바라보는 눈빛이 귀여워서 화가 나기도 했지만, 결국은 이해해주기로 했어.

[PREDICTED DIARY]
오늘 딸이 방이 더러워서 화가 나서 그랬어. 그래서 내가 잘못한 게 없나 싶더라. 딸의 마음이 조금은 이해되지만, 그래도 이렇게 대화가 잘 되는 날이 언제 있을까 싶어! 앞으로는 더 많은 대화를 나누고 소통하는 시간을 가져야겠다. 딸을 위해서라도 이런 작은 일들이 소중하게 느껴져, 정말 행복해 보여서 좋네. 앞으로도 계속 사랑스러운 시간이 많으면 좋겠어!" 하고 다짐했지. 이 순간을 잊지 말자며 오늘도 최선을 다해야겠다는 생각이 들어.
딸에게 항상 고맙다고 말해주고 싶었거든? 그런 말을 듣고 나니 내 마음도 따뜻해졌고, 나도 다시 힘을 내게 됐어.

내가 뭘 어떻게 해야 할지 고민이 많아졌는데, 그 마음을 이해할 수 있었네요. 요즘 애가 많이 성장해서 엄마를 닮았구나 하는 걸 느꼈어요. 아들이 자라나는 모습을 보면서 힘이 나는 것 같아요... 
주변에서 도와주는 것도 중요하지만, 서로 믿고 의지할 때 더욱 큰 행복을 느끼게 돼서, 하루종일 행복한 시간이었음 해줘라. 다음에는 좀 특별하고 따뜻한 마음으로 함께 할게 있어 줘 감사하야. 그렇게 소중한 기억을 간직했으면 한 번쯤 남길래야 한다. 매일매일 너와 함께하는 특별한 날들을 만들어가고 있음이 너무 뿌듯하다. 가족과 지인들과 소소한 일상 속에서 느끼는 기쁨을 잊어버릴
[Sample 2]
[SUMMARY (INPUT)]
1. 아들이 아침 식탁에 앉아 있지만, 음식에 손을

/tmp/ipython-input-1394716629.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  test_trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


[TEST LOSS] 2.5922
[TEST PPL]  13.3585

[INFO] 테스트셋 전체 ROUGE_L 계산 중...

[TEST METRICS] (참고용)
ROUGE-L: 0.0000

[INFO] 테스트 평가지표 저장: /content/drive/MyDrive/aiary/models/day_diary_from_summary_v2/test_metrics_rougeL.json

[INFO] 전체 파이프라인 완료
모델/토크나이저/그래프 저장 위치: /content/drive/MyDrive/aiary/models/day_diary_from_summary_v2

[INFO] 테스트셋 전체 생성 결과 저장 중...
[INFO] CSV 저장 완료: /content/drive/MyDrive/aiary/models/day_diary_from_summary_v2/test_generated_results.csv
[INFO] JSONL 저장 완료: /content/drive/MyDrive/aiary/models/day_diary_from_summary_v2/test_generated_results.jsonl
